In [117]:
import time
from time import sleep
import traceback
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import  Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import re
from openpyxl import load_workbook
from datetime import date
from glob import glob
from shutil import move
import tabula
from PyPDF2 import PdfReader


def loginSEI():
    navegador.get("https://sei.rj.gov.br/sip/login.php?sigla_orgao_sistema=ERJ&sigla_sistema=SEI")

    usuario = navegador.find_element(By.XPATH, value='//*[@id="txtUsuario"]')
    usuario.send_keys(os.environ['login_sefaz'])

    senha = navegador.find_element(By.XPATH, value='//*[@id="pwdSenha"]')
    senha.send_keys(os.environ['senha_sefaz'])

    exercicio = Select(navegador.find_element(By.XPATH, value='//*[@id="selOrgao"]'))
    exercicio.select_by_visible_text('SEFAZ')

    btnLogin = navegador.find_element(By.XPATH, value='//*[@id="Acessar"]')
    btnLogin.click()


    navegador.maximize_window()
    time.sleep(5)

    navegador.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
    time.sleep(1)
    trocarCoordenacao()

def trocarCoordenacao():
    coordenacao = navegador.find_elements(By.XPATH, "//a[@id = 'lnkInfraUnidade']")[1]
    if coordenacao.get_attribute("innerHTML") == 'SEFAZ/COOEGOE':
        coordenacao.click()
        WebDriverWait(navegador,5).until(EC.presence_of_element_located((By.XPATH, "//div[contains(text(), 'Trocar Unidade')]")))
        navegador.find_element(By.XPATH, "//td[text() = 'SEFAZ/COOAJUR' ]").click() 
        
def abrirPastas():
    listaDocs = WebDriverWait(navegador,5).until(EC.presence_of_element_located((By.ID, "divArvore")))
    pastas = listaDocs.find_elements(By.XPATH, '//a[contains(@id, "joinPASTA")]')
    for doc in pastas[:-1]:
        doc.click() 
        sleep(4)
        
def verificarCompetencia():
    docs = navegador.find_elements(By.XPATH, "//div[@id = 'divArvore']//div//a[@class = 'infraArvoreNo']")
    quantDocs = len(docs) 
    for doc in reversed(range(quantDocs)):
            docTexto = docs[doc].text
            if "Despacho de Encaminhamento de Processo" in docTexto:
                docs[doc].click()
                navegador.switch_to.default_content()            
                WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrVisualizacao")))
                WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvoreHtml")))
                body = navegador.find_element(By.TAG_NAME, 'body').text
                if "Conforme despacho retro, solicitamos o processamento do DARJ ora acostado." in body:
                    if "AUDITORA FISCAL" in body.upper() or "AUDITOR FISCAL" in body.upper():
                        return True
                    else:
                        return False
                        
    return False
                


navegador = webdriver.Firefox()
processoSEI = "SEI-140001/034741/2024"

loginSEI()

barraPesquisa = navegador.find_element(By.ID, "txtPesquisaRapida")

barraPesquisa.send_keys(processoSEI)
barraPesquisa.send_keys(Keys.ENTER)

WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvore")))

abrirPastas()

In [114]:
navegador.switch_to.default_content()
WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvore")))


True

In [123]:
navegador.switch_to.default_content()
WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvore")))
docs = navegador.find_elements(By.XPATH, "//div[@id = 'divArvore']//div//a[@class = 'infraArvoreNo']")
quantDocs = len(docs)
for doc in (range(quantDocs)):
    docTexto = docs[doc].text    
    
    if "Ofício" in docTexto:   
        docs[doc].click()
        navegador.switch_to.default_content()            
        WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrVisualizacao")))
        WebDriverWait(navegador,20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "ifrArvoreHtml")))
        time.sleep(2)
        body = navegador.find_element(By.XPATH, '//body').text
        print(body)
        processoJudicial = re.search(r"Ref: (\d[\n*\w*\s\(\)\.,-\/]*)\n\n",body)#DARJ
        if processoJudicial:
            print(processoJudicial.group(1))
        break
           

de 2
Zoom automático
Tamanho real
Ajustar à janela
Largura da página
0%
50%
75%
100%
125%
150%
200%
300%
400%
PROCURADORIA GERAL DO ESTADO

COORDENADORIA GERAL DAS PROCURADORIAS REGIONAIS - 10ªPR CAMPOS

Elaborado por: Ricardo Lima Almeida em 29/07/2024 - PA: PGE/024.002307/2023

1

Ofício PGE/PG11/BLM/ nº_______/2024

10ª PROCURADORIA REGIONAL – CAMPOS DOS GOYTACAZES

(Favor incluir na resposta o número do processo SEI e do PA)

De: 10ª PR-Campos dos Goytacazes

Para: Secretaria de Fazenda do Estado do Rio de Janeiro - SEFAZ

(URGENTE - RESPOSTA DENTRO DE 10 DIAS)

Ref: 0002922-27.2022.8.19.0070

Processo Administrativo: PGE/024.002307/2023

Assunto: Informação de entrada em receita de valores

Trata-se de execução fiscal em curso proposta pelo Estado do Rio de Janeiro em face de
 B.DIAS

DA SILVA PESCADOS
 com o fim de satisfazer a sua pretensão creditícia no que tange ao débito inscrito

em dívida ativa sob a certidão 2021/247.894-3.

Nesta ocasião, solicitamos que seja informado se

: 

In [109]:
from datetime import datetime

hoje = datetime.now().strftime("%d/%m/%Y")
print(hoje)

11/09/2024
